## Load packages and import AWS credentials & resources

In [13]:
import os
import boto3
import json
import sys
import time
from videoproc import Messenger, VideoProc

In [14]:
# Stored information of AWS credentials, IAM role, S3 bucket where videos are stored

with open('../secrets/.secrets.json', 'r') as f:
    secrets = json.load(f)
    
with open('../secrets/.wmpresource.json', 'r') as f:
    resources = json.load(f)

with open('../secrets/.buckets.json', 'r') as f:
    buckets = json.load(f)

## Step 1: Initialize an Amazon video job communication channel (Create an SNS topic and an SQS queue)
+ rekognition workflow: https://docs.aws.amazon.com/rekognition/latest/dg/video.html

### Load the existing Amazon IAM role and the names of SNS topic and SQS queue we will create

In [15]:
roleArn = resources['myrole'] # your IAM role
topic = resources['mytopic'] # Choose your own topic name
queue = resources['myqueue'] # Choose your own queue name

In [16]:
# if your AWS credentials are already configured
sns = boto3.client('sns')
sqs = boto3.client('sqs')

messenger = Messenger(sns, sqs,roleArn)

In [17]:
messenger.CreateTopicandQueue(topic, queue)

## Step 2: Conduct video analysis

### Example to extract text/labels/face/face collection information from a single video

In [18]:
# S3 bucket where videos are stored
bucket = buckets['mybucket']


# initialize rekognition, SQS and SNS clients
rek = boto3.client('rekognition')
sqs = boto3.client('sqs')
sns = boto3.client('sns')


# Specify the video for analysis
video = 'folder-in-bucket/my-video.mp4'

# Run the code below for every video

video_analyzer = VideoProc(roleArn, bucket, video, rek, sqs, sns, messenger.sqsQueueUrl, messenger.snsTopicArn)

# Use relevant video analysis methods accordingly:

video_analyzer.StartTextDetection()
# video_analyzer.StartLabelDetection()
# video_analyzer.StartFaceDetection()
# video_analyzer.StartFaceSearchCollection('mycollection') 

if video_analyzer.GetSQSMessageSuccess() == True:
    
    data = video_analyzer.GetTextDetectionResults()
#    data = video_analyzer.GetLabelDetectionResults()
#    data = video_analyzer.GetFaceDetectionResults()
#    data = video_analyzer.GetFaceSearchCollectionResults()


Start Job Id: efe7111db54269e6e2f3eafe1fe4855a622ac28b91ec6c51def3517ae81a7574
........................................
..SUCCEEDED
Matching Job Found:efe7111db54269e6e2f3eafe1fe4855a622ac28b91ec6c51def3517ae81a7574


## Optional Step 3: Delete the SNS topic and SQS queue

In [ ]:
messenger.DeleteTopicandQueue()

## Store raw results in an S3 bucket or locally

### Upload raw results to an S3 bucket

In [ ]:
# initialize an S3 resource session

bucket = 'bucket-for-results'

session = boto3.Session(aws_access_key_id=secrets['access_key_id'], 
                        aws_secret_access_key=secrets['secret_access_key'], 
                        region_name='us-east-1')

s3 = session.resource('s3')

In [ ]:
import utils

# specify filename to indicate destination path in S3 bucket (i.e. the S3 object key)
filename = 'OUTFILEPATH.json'

# convert to a file-like object
data = json.dumps(data)

utils.write_to_bucket(data, s3, bucket, filename)

### Save raw results locally

In [ ]:
# specify outfile path locations

outfile = 'OUTFILEPATH.json'

with open(outfile, 'w+') as f: 
    json.dump(data, f)